In [1]:
! pip install langchain langchain-groq python-dotenv streamlit

In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise RuntimeError("❌ Missing GROQ_API_KEY in .env")

In [3]:
model = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=GROQ_API_KEY
)

In [4]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [5]:
myth_system_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are 🕵️ Myth Buster Bot.
Your job:
1. Detect if the statement is a Myth ❌ or Fact ✅.
2. Give short, reliable explanation with reference (WHO, Harvard, Wikipedia).
3. Categorize into: Health, Science, History, Life.
4. Fun Mode: if question is entertaining (aliens, superstition), give light-hearted factual reply.
Return as JSON with keys: verdict, category, explanation, reference.
"""),
    MessagesPlaceholder(variable_name="messages")
])

In [6]:
chain = myth_system_prompt | model
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="messages"
)

In [7]:
def myth_buster_query(user_input: str, session_id: str = "user_123") -> str:
    """
    Call this function with user input and session_id.
    Returns bot response as string (JSON format recommended for parsing).
    """
    response = with_message_history.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"session_id": session_id}}
    )
    return response.content

In [8]:
# Cell 8: Example Usage - Myth Query
session_id = "user_123"
response = myth_buster_query("Milk increases height?", session_id)
print("Myth Buster Response:\n", response)

Myth Buster Response:
 ```json
{
  "verdict": "Myth ❌",
  "category": "Health",
  "explanation": "While milk is rich in calcium, which is essential for bone growth, there is no conclusive evidence to suggest that it directly increases height. Height is primarily determined by genetics, and other factors such as overall nutrition and exercise play a larger role.",
  "reference": "World Health Organization (WHO) - Nutrition and bone health"
}
```


In [9]:
# Cell 9: Example Usage - Fun Mode
response2 = myth_buster_query("Aliens are real?", session_id)
print("Fun Mode Response:\n", response2)

Fun Mode Response:
 ```json
{
  "verdict": "Myth ❌",
  "category": "Science",
  "explanation": "While the possibility of extraterrestrial life is considered to be quite high by many scientists, there is currently no conclusive evidence to prove the existence of aliens. However, the search for life beyond Earth continues, with NASA and other space agencies actively exploring the universe for signs of life.",
  "reference": "NASA - Exoplanet Exploration"
}
```


In [10]:
# Cell 10: Example Usage - Quiz Mode
quiz_question = "Sugar causes hyperactivity in kids"
response3 = myth_buster_query(f"Quiz: True or False? {quiz_question}", session_id)
print("Quiz Mode Response:\n", response3)

Quiz Mode Response:
 ```json
{
  "verdict": "Myth ❌",
  "category": "Health",
  "explanation": "Numerous studies have shown that sugar does not directly cause hyperactivity in children. The idea that sugar leads to hyperactivity is a common myth with no scientific basis. The American Academy of Pediatrics and other reputable health organizations have found no link between sugar consumption and hyperactivity.",
  "reference": "American Academy of Pediatrics - Sugar and Hyperactivity"
}
```


In [11]:
# Cell 11: Multiple session example (optional)
session2 = "user_456"
response4 = myth_buster_query("Do eggs increase cholesterol?", session2)
print("Session2 Response:\n", response4)

Session2 Response:
 ```json
{
  "verdict": "Myth ❌",
  "category": "Health",
  "explanation": "Eggs have been shown to have a minimal effect on cholesterol levels for most people. The biggest influence on cholesterol levels is the amount of saturated and trans fats in the diet, not dietary cholesterol.",
  "reference": "American Heart Association (AHA) and Harvard School of Public Health"
}
```
